<div align="center"><h1><b><font size="6">
  Clarification of Transport Sector Input Assumptions for the négaWatt-BE Model</h1></font></div>
<div align="center"><h2><b><font size="5">
  DEMAND OF SEVERAL SECTOR IN BELGIUM
</h2></font></b></div>

---

<div align="center"><h3><font size="3">
  <b> Developers:</b> BERNAERTS Valentine
</h3></font></div>

---

<font size="5"><b>Table of Contents</b></font>
<a class="anchor" id="contents"></a> <br>
1. [Introduction](#section_1)
2. [Input Table (test)](#section_2)  
3. [Total Passenger Demand](#section_3)   
   1. [Input](#section_3_1)       
   2. [Graphs](#section_3_2)
      1. [Bar chart](#section_3_2_1)  
      2. [Pie chart](#section_3_2_2)  
4. [Modal Share per Area](#section_4)  
   1. [Urban Modal Share](#section_4_1)   
   2. [Rural Modal Share](#section_4_2)    
   3. [Graphs](#section_4_3)  
      1. [Bar chart](#section_4_3_1)  
      2. [Pie chart](#section_4_3_2)   
5. [Vehicle](#section_5)  
   1. [Vehicle Occupancy](#section_5_1)  
   2. [Vehicle Stock](#section_5_2)  
   3. [Vehicle Technology Parameters](#section_5_3)
      1. [Energy Consumption [Wh/pkm]](#section_5_3_1)
      2. [Investment and Maintenance Cost](#section_5_3_2)  
      3. [Lifetime](#section_5_3_3)  
      4. [Speed assumptions](#section_5_3_4)  
   4. [Graphs](#section_5_4)  
      1. [Bar chart](#section_5_4_1)  
      2. [Pie chart](#section_5_4_2)    
6. [Infrastructure Data](#section_6)      
   1. [Input](#section_6_1)
      1. [Road](#section_6_1_1)
      2. [rail](#section_6_1_2)  
      3. [charging stations](#section_6_1_3)
   2. [Cost and sizing assumptions](#section_6_2)
   3. [Graphs](#section_6_3)   
      1. [Bar chart](#section_6_3_1)  
      2. [Pie chart](#section_6_3_2)     
8. [Fuel Types & Supply Constraints](#section_7)  
   1. [Fuel Types](#section_7_1)  
   2. [Supply Constraints](#section_7_2)  
   3. [Graphs](#section_7_3)  
      1. [Bar chart](#section_7_3_1)  
      2. [Pie chart](#section_7_3_2)   
9. [Model output](#section_9)  
    1. [Final dataframe](#section_9_1)   
    2. [Download dataframe](#section_9_2)   
    3. [Graphs](#section_9_3)  
       1. [Bar chart](#section_9_3_1)
10. [References](#section_10)   

<font size="5"><b>Rappel pour moi</b></font>

Documenter précisément et de manière lisible toutes les hypothèses d’entrée (inputs) du secteur du transport, en partant de ce qui est déjà fait par Damien (PDF + Excel), en identifiant:
<ul>
 <li> Quelle est l'hypothèse utilisée ?</li>
 <li> Quelle est la valeur entrée dans le modèle ?</li>
 <li> D’où vient-elle (quel model) ?</li>
 <li> Pourquoi a-t-elle été choisie / modifiée ?</li>
</ul>

<font size="5"><b>Acronyms</b></font>
*   <b>DS</b>: Diesel
*   <b>EI</b>: Energy Intensity
*   <b>EL</b>: Electricity
*   <b>EUD</b>: End-use demand
*   <b>FEC</b>: Final energy consumption
*   <b>FG</b>: Fossil gas or natural gas
*   <b>GS</b>: Gasoline
*   <b>HVC</b>: High value chemicals
*   <b>LFO</b>: Light fuel oil
*   <b>LPG</b>: Liquefied petroleum gas
*   <b>MSMEs</b>: Small and medium-sized enterprises
*   <b>RES_SOLAR</b>: Solar resource
*   <b>SUV</b>: Sport utility vehicle
*   <b>WD</b>: Biomass


---

<font size="7"><b>SECTION TRANSPORT</b></font> 

---


---

<font size="5"><b>1. Introduction</b></font> <a class="anchor" id="section_1"></a>

---

To ensure transparency and coherence in the energy modelling process of the négaWatt-BE project, this notebook clarifies all key input assumptions related to the transport sector. It aims to trace the origin of each numerical input and justify its relevance or adaptation.

The current documentation is based on the detailed work by Damien Carbonnelle and the input data of the projet negaWatt, particularly:
<ul>
 <li> His Master's thesis: *"What are the energy, fleet, and economic impacts of different passenger land-based mobility scenarios in 2050 on Belgium’s energy system?"* (UCLouvain, 2025)</li>
 <li> His associated Excel dataset: `Mobility analysis.xlsx`</li>
 <li> The dashboard of Belgium for the project : `Dashboard_BE_v5.27.xlsx`</li>
 <li> (The structure and logic from the notebook `Characterization of Bolivian energy demand.ipynb`, used as inspiration for formatting and approach.)</li>
</ul>
Additional references will probably be added futher.


> Attention de bien rappeler qu'on est dans le cas "Modal shares without airplane (assumptions made WITH soft modes as they participate in the shift of car kilometers)


---

<font size="5"><b>2. Input table (test)</b></font> <a class="anchor" id="section_2"></a>

---

In [1]:
import time
import pandas as pd
import xarray as xr
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from dataclasses import dataclass
import math
import random


# Loading the Excel file
file_path = 'data/Mobility analysis.xlsx' #file_path = '../data/Mobility analysis.xlsx'
input_mobility = pd.read_excel(file_path, sheet_name='InputOutput-mobility', header=None) 
calculation_mobility = pd.read_excel(file_path, sheet_name='Calculation-mobility', header=None) 
output_mobility = pd.read_excel(file_path, sheet_name='Output_Keyfindings', header=None)

# Extracting the first table
tableau_1 = input_mobility.iloc[3:17, 0:10]
tableau_1.columns = ['Sufficiency parameters', 'Unit', '2019', '2025', '2030', '2035', '2040', '2045', '2050', 'sources']  # rename columns
tableau_1 = tableau_1.fillna('') # Replace NaN values with blank space

display(tableau_1)

,Sufficiency parameters,Unit,2019,2025,2030,2035,2040,2045,2050,sources
3,Distance,[Gpkm/year],141.719,142.63,133.159,131.047,121.634,115.885,110.655,Assumptions_Transport_nWBE_220630
4,Modal shift,[%],0.801179,0.783477,0.747241,0.699663,0.6373,0.597561,0.568235,Assumptions_Transport_nWBE_220630
5,Lifetime,[Years],11.9,11.9,11.9,11.9,11.9,11.9,11.9,https://plateforme-wallonne-giec.be/lettre-37
6,Average annual distance travelled by car,[km/year],15151,15151.0,15151.0,15151.0,15151.0,15151.0,15151.0,5v_27
7,Car occupancy,[people/car],1.25,1.4,1.72,1.82,1.9,1.95,2.0,5v_27
8,Speed limit & sustainable driving gain,[%],0,0.0,-2.0,-2.0,-2.0,-2.0,-2.0,5v_27 and negawatt fr
9,Car design,[%],0,0.0,0.0,-0.5,-1.0,-1.5,-2.0,5v_27
10,Techno parameters,,,,,,,,,
11,Electrification,[%],0.5,6.0,18.0,35.0,56.0,78.0,99.5,Assumptions_Transport_nWBE_220630
12,Efficiency,[kWh/km],0.22,0.213,0.19,0.161,0.145,0.128,0.128,5v_27 = negawatt fr



---

<font size="5"><b>3. Base Assumptions (2025)</b></font> <a class="anchor" id="section_3"></a>

---

To reduce the final energy demand from the transport sector, two complementary strategies are considered: sufficiency and efficiency.
These strategies are structured into six levers, which guide the transition toward a low-energy mobility system.

Sufficiency refers to a reduction in the need for energy-intensive transport activity. It is achieved through behavioral and systemic changes that avoid or reduce car use, including modal shifts and shared mobility.

Efficiency refers to technical improvements that reduce the amount of energy consumed per kilometre travelled or per vehicle, without necessarily reducing mobility demand.



<div align="justify"> <font size="3">
<b>References:</b></font>
    
* [1] Carbonnelle, D. (2025). <i>What are the energy, fleet, and economic impacts of different passenger land-based mobility scenarios in 2050 on Belgium’s energy system?</i> UCLouvain. Available at: <a href='https://thesis.dial.uclouvain.be/server/api/core/bitstreams/9e2fb17d-7123-47e0-93e4-6439db615d63/content'>thesis.dial.uclouvain.be</a>
* [2] Carbonnelle, D. (2025). <i>Mobility analysis.xlsx</i>. Dataset available at: <a href='https://docs.google.com/spreadsheets/d/1OnB7Udk3wH8eiy5hMRUjauWr3yoOTl7n/edit#gid=10993834'>Google Sheets</a>
* [3] CLEVER Project Consortium (June 2023). <i>CLEVER – Climate neutrality, Energy security and Sustainability: A pathway to bridge the gap through Sufficiency, Efficiency and Renewables</i>. Available at: <a href='https://clever-energy-scenario.eu/'>clever-energy-scenario.eu</a>
* [4] Meyer, S. (2022). <i>Assumptions_Transport_nWBE_220630</i>. Internal documentation of transport assumptions for négaWatt France, adapted for the Belgian context. Available at: <a href='https://cloud.negawatt.be/s/YjD5Rioe954yXTS?dir=undefined&path=%2FFichiers%20source%20Seb&openfile=22735'>cloud.negawatt.be</a>
* [5] PyPSA-Eur Belgium Team (2025). <i>Dashboard_BE_v5.28</i>. Technical assumptions and data for Belgium adapted from the PyPSA-Eur model, including several transport parameters based on négaWatt France sources. Available at: <a href='https://docs.google.com/spreadsheets/d/18B1CIOUUL_SguTTWCsPzmzdZ5GC4KarZ/edit#gid=1351106561'>Google Sheets</a>
</div>

#### Population:
The total Belgian population is set at 11,507,000 inhabitants for the base year. 
This value originates from Statbel, Belgium’s official statistics office, and was corrected and harmonised by négaWatt France using Eurostat projections as part of the CLEVER project [3].
The figure is directly retrieved from the Dashboard_BE_v5.28.xlsx file [5].

> Note: Although the reference year is labelled as 2020, most input data are taken from 2019, due to the disruptions caused by the COVID-19 pandemic in 2020. This choice ensures greater consistency and realism in the baseline and future projections.

In [2]:
population_20 = 11507000  # [3] Total population in Belgium in 2020
population_30 = 11757990  # [3] Total population in Belgium in 2030
population_40 = 11894881  # [3] Total population in Belgium in 2040
population_50 = 11926987  # [3] Total population in Belgium in 2050


---

<font size="4"><b>3.1. How They Affect Final Energy Demand </b></font> <a class="anchor" id="section_3_2"></a>

---

All six levers act on a **bottom-up calculation** of energy demand:

$$
\textcolor{orange}{\text{Final Energy}} = \sum_{\text{modes}} \left( \frac{\textcolor{orange}{\text{pkm}}}{\textcolor{orange}{\text{occupancy}}} \times \textcolor{blue}{\text{energy per vkm}} \times \textcolor{blue}{\text{fuel share}} \right)
$$

Each lever modifies a part of this equation:

#### Sufficiency:

  * `Travelled distance` → acts on **pkm**
  * `Modal shift` → redistributes **pkm per mode**
  * `Car occupancy` → affects **vehicle-km (vkm)**
  * `Speed limit` → improves **vehicle efficiency**

#### Efficiency:

  * `Fuel switch` → improves **conversion efficiency**
  * `Vehicle efficiency` → reduces **energy per vkm**



---

<font size="4"><b>3.2. Sufficiency </b></font> <a class="anchor" id="section_3_1"></a>

---

#### a) Travelled Distance (Gpkm/year) 

The travelled distance represents the total number of kilometres covered annually by a person using all land-based transport modes. It includes commuting, leisure, and other mobility needs.

In 2020, this value amounts to **151,9 Gpkm/year** [4], which corresponds to an average of **13200 km/person/year**. 

$\textcolor{Orange}{\textbf{Assumption:}}$ In line with typical negaWatt france friends (2017), a **15% reduction** is expected by 2050, driven by the expansion of teleworking and a higher share of urban housing. This leads to an estimated **130,18 Gpkm/year** , equivalent to **10915 km/person/year** [4] in 2050. 


#### b) Modal Shift

The travelled distance is the total number of kilometres covered by a person in a year using all land-based transport modes, including soft mobility options such as walking and cycling.

In 2020, **74,75%** of all passenger-kilometres are made by car [4], while the remaining 25.25% are shared among public transport, cycling, walking, and other modes.

$\textcolor{Orange}{\textbf{Assumption:}}$ The objective is to **reduce car modal share to 48,3%** by 2050 [4] through increased use of soft and collective mobility solutions.


#### c) Car Occupancy

Car occupancy refers to the average number of people travelling in a car simultaneously. It directly impacts the number of vehicle-kilometres required to meet passenger demand.

In 2020, the average occupancy rate is **1.25 persons per car** [5].

$\textcolor{Orange}{\textbf{Assumption :}}$ With the development of carpooling and shared mobility services, this figure is expected to increase to **rise it to 2 persons per car** by 2050 [5].


#### d) Speed Limit & Driving Behaviour 

In 2020, no specific policies were implemented to reduce speeds or encourage eco-driving. The reference energy efficiency gain is therefore set to **0%** in the base year. 

However, smoother driving and lower speed limits reduce aerodynamic drag and rolling resistance, contributing to reduced energy consumption per kilometre.

$\textcolor{Orange}{\textbf{Assumption:}}$ From 2030, new speed regulations will be introduced across all regions: 
- 110 km/h on highways (currently 120 km/h),
- 70 km/h on regional roads,
- 30 km/h in urban areas.
According to the négaWatt France scenario, reducing highway speed from 130 km/h to 110 km/h, combined with more sustainable driving behaviour, leads to a 3% reduction in fuel consumption. In Belgium, since the initial limit is already 120 km/h, the expected reduction in energy use is estimated at **2%** [2].

This change is assumed to occur in a single step in 2030, and its impact is not explicitly modelled, but rather approximated within the global vehicle efficiency improvement described in the next section.

$\textcolor{red}{\textbf{ATTENTION :}}$ This reduction seems really small. It should be checked again for a better reference!

#### e) Vehicle Design (Weight Reduction)
The evolution of vehicle design, especially the reduction in average weight, contributes to lower energy consumption per kilometre.

$\textcolor{Orange}{\textbf{Assumption:}}$ From 2035, a progressive decrease in vehicle mass will lead to a **2% reduction** in energy consumption by 2050 [2].

Like speed policies, this effect is not explicitly isolated in the model but is approximated as part of the global efficiency improvement of electric vehicles.

$\textcolor{red}{\textbf{ATTENTION :}}$ The assumption of a 2% energy reduction by 2050 due to vehicle weight is taken from Mobility analysis.xlsx [2], but does not appear in any other confirmed source. It is loosely attributed to négaWatt France, but lacks clear justification. Further validation is recommended.

> All parameters above are loaded in the next Python cell. Any of them can be adjusted manually to test other assumptions.


In [3]:
# --- BASE YEAR 2020 – SUFFICIENCY INPUTS ---  -> panda dataframe

passenger_km_20 = 151.9                                     # Gpkm/year [4] --> Total demand
km_per_person_20 = passenger_km_20 / population_20 * 10**6  # km/person/year 
car_modal_share_20 = 0.7475                                 # [4]
car_occupancy_20 = 1.25                                     # persons/car [5]
speed_limit_gain_20 = 0.00                                  # energy gain due to speed/driving [2]
weight_reduction_20 = 0.00                                  # energy gain due to weight reduction [2]


# --- PROJECTION – SUFFICIENCY INPUTS ---   -> faire la fonction en "s"? quel courbe demander à m. quoilin comment il le voit et contact pour la fonction en s envoyé par mail par sébastien 

passenger_km_30 = 146.97                                    # Gpkm/year [4] --> Total demand
km_per_person_30 = passenger_km_30 / population_30 * 10**6  # km/person/year
car_modal_share_30 = 0.6770                                 # [4]
car_occupancy_30 = 1.72                                     # persons/car [5]
speed_limit_gain_30 = 0.02                                  # energy gain due to speed/driving [2]
weight_reduction_30 = 0.00                                  # energy gain due to weight reduction [2]

passenger_km_40 = 139.17                                    # Gpkm/year [4] --> Total demand
km_per_person_40 = passenger_km_40 / population_40 * 10**6  # km/person/year
car_modal_share_40 = 0.5570                                 # [4]
car_occupancy_40 = 1.90                                     # persons/car [5]
speed_limit_gain_40 = 0.02                                  # energy gain due to speed/driving [2]
weight_reduction_40 = 0.01                                  # energy gain due to weight reduction [2]


# --- GOAL 2050 – SUFFICIENCY INPUTS ---

passenger_km_50 = 130.18                                    # Gpkm/year [4] --> Total demand
km_per_person_50 = passenger_km_50 / population_50 * 10**6  # km/person/year
car_modal_share_50 = 0.483                                  # [4]
car_occupancy_50 = 2                                        # persons/car [5]
speed_limit_gain_50 = 0.02                                  # energy gain due to speed/driving [2]
weight_reduction_50 = 0.02                                  # energy gain due to weight reduction [2]



---

<font size="4"><b>3.2. Sufficiency </b></font> <a class="anchor" id="section_3_1"></a>

---

#### a) Fuel Switch (Electrification)

In 2020, only **0.5%** of the car fleet in Belgium is electrified [2]. Most vehicles still run on fossil fuels (diesel, petrol, CNG, etc.).

$\textcolor{Orange}{\textbf{Assumption:}}$ By 2050, the electrification rate is expected to reach **99.5%** [2] of the fleet. This implies a complete phase-out of combustion engines, progressively replaced by electric vehicles. The transition is assumed to be linear over the period.

#### b) Energy Efficiency Improvement

In 2020, the average specific energy consumption for electric vehicles (BEVs) in Belgium is set at **0.22 kWh/km** [5], representing the combined result of current vehicle design, driving habits, and technical performance.

$\textcolor{Orange}{\textbf{Assumption:}}$ By 2050, this value is expected to decrease to **0.13 kWh/km**, based on projections from the négaWatt France scenario. This value is directly used in the Belgian dashboard [5], assuming that vehicle efficiency evolves similarly in Belgium and France, without further national adaptation.

This overall improvement of 40% reflects the combined effect of several technological changes:
– lighter vehicles,
– improved aerodynamics,
– regenerative braking,
– more efficient powertrains and battery systems.
- Gradual adoption of speed-limiting policies and eco-driving behaviour

$\textcolor{red}{\textbf{NOTE :}}$ Although these improvements implicitly account for the effects of reduced speed and vehicle weight, they are not modelled separately. Future work could aim to link these effects more transparently to this parameter.

In [ ]:
# --- BASE YEAR 2020 – EFFICIENCY INPUTS ---

electrification_20 = 0.005       # 0.5% of the car fleet is electric [2]
efficiency_20 = 0.220            # kWh/km for electric vehicles [5]


# --- PROJECTION – EFFICIENCY INPUTS ---  -> courbe en "s"?

electrification_30 = 0.18        # 34% of the fleet is electric [2]
efficiency_30 = 0.190            # kWh/km [5]

electrification_40 = 0.56        # 84% [2]
efficiency_40 = 0.145            # kWh/km [5]


# --- GOAL 2050 – EFFICIENCY INPUTS ---

electrification_50 = 0.995       # 99.5% of the fleet is electric [2]
efficiency_50 = 0.3            # kWh/km [5]



---

<font size="5"><b>4. Modal shares</b></font> <a class="anchor" id="section_4"></a>

---


...

In [5]:
import pandas as pd  # hisolé toutes les hypothéses donc de réduction de pourcentage, et puis car = 100- toutes les hypothése des autre mode

# 1) Load & slice
file_path = 'data/Assumptions_Transport_nWBE_220630.xlsx'
modal_shares = pd.read_excel(file_path, sheet_name='Passengers', header=None)
tab = modal_shares.iloc[14:38, 0:12]
tab.columns = ['Mode','Unit','2015','2017','2019','2025','2030','2035','2040','2045','2050','comment']
tab = tab.drop(columns=['2015','2017']).reset_index(drop=True)
tab = tab.fillna('').infer_objects(copy=False) # Replace NaN values with blank space

# 2) Only fill missing comments (no global fillna)
tab['comment'] = tab['comment'].fillna('')

# 3) Identify the year-columns
numeric_cols = ['2019','2025','2030','2035','2040','2045','2050']

# 4) Highlighting functions
def highlight_key(val):
    try:
        v = float(val)
        if abs(v - 100.0) < 0.01:
            return 'background-color: #ffe5b4; font-weight: bold'  # light beige
        if abs(val - 264) < 0.1 or abs(val - 327.45) < 0.1 or abs(val - 620.4) < 0.1 or abs(val - 1309.8) < 0.1 or abs(val - 74.75) < 0.1 or abs(val - 48.3) < 0.1: 
            return 'background-color: #ffcccc; font-weight: bold'  # pale red
        if abs(val - 151.9) < 0.1 or abs(val - 130.18) < 0.1 or abs(val - 13200) < 0.1 or abs(val - 10914.99) < 0.1: 
            return 'background-color: #ffcccc; font-weight: bold'  # pale red
    except:
        pass
    return ''

def highlight_lines(row):
    m = row['Mode']
    if m == 'TOTAL':
        return ['border-top: 4px double black'] * len(row)
    if m in ['Pedestrian','Bicycle','Motorcyc','Car','Bus/coach','Tram/metro','Train']:
        return ['border-top: 2px solid black'] * len(row)
    return [''] * len(row)

# 5) Build styled table
styled = (
    tab.style
       .map(highlight_key, subset=numeric_cols)           # use .map, not .applymap
       .apply(highlight_lines, axis=1)
       .set_properties(subset=['Unit'], **{'font-style':'italic','color':'gray'})
       .set_properties(subset=['Mode'], **{'font-weight':'bold'})
       .format({c:"{:.2f}" for c in numeric_cols})       # 2 decimals
       .set_table_attributes('style="width:100%;table-layout:fixed;"')
       .set_table_styles([
           {'selector':'th, td', 'props':[('padding','8px 12px')]}
       ])
       .set_caption("Modal Shares – whitout airplane and with soft modes.")
       .hide(axis='index')
)

display(styled)


C:\Users\valen\AppData\Local\Temp\ipykernel_13076\3662130659.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  tab = tab.fillna('').infer_objects(copy=False) # Replace NaN values with blank space


Mode,Unit,2019,2025,2030,2035,2040,2045,2050,comment
Pedestrian,% du total,2.00,2.20,2.40,2.60,2.80,2.90,3.00,
,km/per,264.00,281.60,300.00,318.50,327.60,327.70,327.45,
,Gpkm,3.04,3.39,3.53,3.83,3.90,3.90,3.91,
Bicycle,% du total,4.70,5.20,7.00,8.50,9.80,11.00,12.00,
,km/per,620.40,665.60,875.00,1041.25,1146.60,1243.00,1309.80,
,Gpkm,7.14,8.01,10.29,12.53,13.64,14.81,15.62,
Motorcyc,% du total,1.30,1.60,1.90,2.20,2.60,3.00,3.50,
,km/per,171.60,204.80,237.50,269.50,304.20,339.00,382.02,
,Gpkm,1.97,2.46,2.79,3.24,3.62,4.04,4.56,
Car,% du total,74.75,72.55,67.70,62.20,55.70,51.45,48.30,



---

<font size="4"><b>5.1. Industry Parameters </b></font> <a class="anchor" id="section_3_3"></a>

---
-> pris du projet negawatt france... trouver la source mettre direct les datas en codes:

In [6]:
# --- Industry parameters [2] ---

weight = 1250                # kg
steel = 68                   # %
curved_glas = 1.9            # %

gasoline = 9,21              # [kWh/L]
diesel = 10,17               # [kWh/L]	
LPG	= 6,69                   # [kWh/L]	
NVG_per_biomethane = 11,3    # [kWh/kg]
hydrogen = 33,3              # [kWh/L]	



---

<font size="4"><b>4.1. Data Sources and Value for 2020 </b></font> <a class="anchor" id="section_4_1"></a>

---


---

<font size="4"><b>4.2. Assumptions and Drivers </b></font> <a class="anchor" id="section_4_2"></a>

---


---

<font size="4"><b>4.3. Calculation: Total pkm </b></font> <a class="anchor" id="section_4_4"></a>

---


---

<font size="4"><b>4.4. Sensitivity Analysis / Graphs </b></font> <a class="anchor" id="section_4_4"></a>

---


---

<font size="5"><b>3. Total Passenger Demand</b></font> <a class="anchor" id="section_3"></a>

---


---

<font size="4"><b>3.1. Input</b></font> <a class="anchor" id="section_3_1"></a>

---

In [7]:
#


---

<font size="4"><b>3.2. Graphs</b></font> <a class="anchor" id="section_3_2"></a>

---

<font size="3"><b>3.2.1. Bar chart - Total Passenger Demand</b></font> <a class="anchor" id="section_3_2_1"></a>